<a href="https://colab.research.google.com/github/xthiagolima/tradutor-de-pptx/blob/main/Tradu%C3%A7%C3%A3o_de_PPTX_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌍 **Traduza apresentações PowerPoint para qualquer idioma com o Gemini** 🚀

---

* ⏱️ **Traduza em segundos ou minutos**. Não perca seu dia com processos manuais.
* 📊 **4, 20, 100 ou mais slides?** Vai tranquilo!
* 🌐 **Suporte a múltiplos idiomas** com alta precisão
* 🎯 **Preserva a formatação original**, layout e posições dos textos intactos
* 🔁 **Revisão automática** em caso de discrepâncias na tradução
* 🤖 **Processo 100% automatizado** com uso de agentes inteligentes
* 🧩 **Integração simples** com documentos PowerPoint (.pptx)
* 💼 Ideal para **equipes globais, apresentações corporativas e conferências multilíngues**

Instalar bibliotecas

In [ ]:
%pip -q install google-genai python-pptx google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

Configura a API Key do Google Gemini

In [ ]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

Importações necessárias para a construção dos agentes utilizando o ADK (Agent Development Kit) do Google.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

### Agente Tradutor 🧠
Recebe uma lista de palavras ou frases e as traduz para o idioma desejado. O prompt engineering é extremamente importante e sensível nesse processo, pois buscamos precisão na correspondência entre os elementos: a quantidade de itens na lista traduzida deve ser exatamente igual à da lista original.

In [ ]:
##########################################
# --- Agente 1: Tradutor --- #
##########################################
def agente_tradutor(lista, idioma):

    qtd = len(lista)

    tradutor = Agent(
        name="agente_tradutor",
        model="gemini-2.0-flash",
        instruction=f"""Você deve traduzir exatamente {qtd} frases para {idioma}.

          Regras obrigatórias:
          1. Mantenha exatamente {qtd} itens na saída
          2. Preserve todos os espaços antes e depois do texto
          3. Mantenha a numeração original
          4. Não adicione ou remova itens
          5. Não adicione comentários ou explicações
          6. Preserve as aspas simples em cada item
          7. Não combine ou divida frases

          Formato de entrada:
          {qtd} frases numeradas, uma por linha, entre aspas simples

          Formato de saída:
          {qtd} frases traduzidas, numeradas, uma por linha, entre aspas simples

          Validação:
          - Conte o número de linhas antes de responder
          - Verifique se cada linha mantém as aspas simples
          - Compare a formatação com o exemplo

          Exemplo de entrada:
          1. 'O carro '
          2. 'era'
          3. ' muito rápido.'

          Exemplo de saída esperada:
          1. 'The car '
          2. 'was'
          3. ' very fast.'

          Antes de finalizar, verifique:
          - Número de itens: {qtd}
          - Formato consistente
          - Preservação dos espaços

        """,
        description="Agente que traduz uma lista de palavras/frases para outro idioma",
    )

    # Prompt do Tradutor
    entrada_do_agente_tradutor = "\n".join(f"{i+1}. '{phrase}'" for i, phrase in enumerate(lista))

    # Executa o agente
    resposta = call_agent(tradutor, entrada_do_agente_tradutor)

    # Converte a resposta em lista
    lines = resposta.split('\n')[:-1]

    i = 1
    lista_traduzida = []
    # Adapta a lista removendo a numeração
    for line in lines:
        line = line.replace(f"{i}. ","")
        line = line.strip("'")
        lista_traduzida.append(line)
        i += 1

    return lista_traduzida

### Agente Revisor ✅
Caso a lista traduzida apresente um número diferente de elementos em relação à original, o Agente Revisor entra em ação. Essa divergência pode causar deslocamentos nos slides, já que os itens traduzidos podem ser posicionados incorretamente. O Revisor tenta garantir que a estrutura original da lista seja preservada após a tradução.

In [ ]:
######################################
# --- Agente 2: Revisor --- #
######################################
def agente_revisor(lista_original, lista_traduzida, idioma):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um revisor que corrige a quantidade de palavras/frases numa lista após traduzida.
            Verifica se algumas palavras traduzidas foram unidas ou separadas.
            Retorne a lista corrigida mantendo a estrutura, a quantidade e sem comentários.
            """,
        description="Agente revisor de itens traduzidos"
    )

    lista_original = "\n".join(f"{i+1}. '{phrase}'" for i, phrase in enumerate(lista_original))

    # Prompt do Revisor
    entrada_do_agente_revisor = f"""

    Lista original ({len(lista_original)} elementos):
    {lista_original}

    Lista traduzida ({len(lista_traduzida)} elementos):
    {lista_traduzida}"""

    # Executa o agente
    resposta = call_agent(revisor, entrada_do_agente_revisor)
    resposta = resposta.replace("```", "")

    import ast
    # Converte para lista a resposta em string
    converter_lista = ast.literal_eval(resposta)

    return converter_lista

### Manipulação de slides pela biblioteca python-pptx
Em cada slide, é realizada uma iteração que verifica se cada *shape* é do tipo texto. Os textos identificados são adicionados a uma lista chamada `full_text`.
Essa lista é então enviada ao **Agente Tradutor**. Caso o número de elementos da lista traduzida não corresponda ao da original, ambas são encaminhadas ao **Agente Revisor**, que realiza a comparação e corrige as diferenças.
Dessa forma, cada elemento é reposicionado corretamente no slide. Esse processo se repete até que todos os slides tenham sido processados.

In [ ]:
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE

def processar_pptx(file, idioma):

  # Carregando o documento
  document = Presentation(file)
  print("Arquivo pptx selecionado")

  # Salve o numero de slides
  num_slides = len(document.slides)
  pag = 0
  failed = []

  # Iterando em cada slide
  for slide in document.slides:
    pag += 1
    print(f"\nTraduzindo slide {pag}")

    full_text = []
    # Iterando em cada shape do slide
    for shape in slide.shapes:
      # Verifica se o shape é do tipo texto
      if shape.has_text_frame:
          for paragraph in shape.text_frame.paragraphs:
              for run in paragraph.runs:
                  full_text.append(run.text)
      # Verifica se o shape é do tipo grupo e itera em cada shape do grupo
      elif shape.shape_type == MSO_SHAPE_TYPE.GROUP:
          for shape in shape.shapes:
              if shape.has_text_frame:
                  for paragraph in shape.text_frame.paragraphs:
                      for run in paragraph.runs:
                          full_text.append(run.text)
      # Verifica se o shape é do tipo tabela
      elif shape.shape_type == MSO_SHAPE_TYPE.TABLE:
          table = shape.table
          for row in table.rows:
              for cell in row.cells:
                  full_text.append(cell.text)

    print(f'Extraído: {len(full_text)}')
    print(full_text)

    try:
      # Enviar lista para o Agente Tradutor
      translated_list = agente_tradutor(full_text, idioma)
      print(f'Traduzido: {len(translated_list)}')
      print(translated_list)

      tentativa = 1
      qtd = len(translated_list)

      # Se houver divergências executa o Agente Revisor
      while tentativa <= 3 and len(full_text) != qtd:
        print("Erro: O número de frases traduzidas não corresponde ao número de frases extraídas.")
        failed.append(pag)
        translated_list = agente_revisor(full_text, translated_list, idioma)
        qtd = len(translated_list)
        print(f'Revisado: {qtd}')
        print(translated_list)
        tentativa += 1

      # Pula o slide caso o Agente Revisor não consiga resolver
      if len(full_text) != len(translated_list):
        print(f"Não foi possível traduzir o slide {pag}")
        continue

      # Retorna as palavras traduzidas iterando novamente os shapes
      # ainda no mesmo slide
      i = 0
      for shape in slide.shapes:
          if shape.has_text_frame:
              for paragraph in shape.text_frame.paragraphs:
                  for run in paragraph.runs:
                      run.text = run.text.replace(run.text,translated_list[i]).replace("\\n","\n").replace("\\'","\'").replace("\\u200b","").replace("\\xa0","").replace("\\t","  ")
                      i += 1
          elif shape.shape_type == MSO_SHAPE_TYPE.GROUP:
              for shape in shape.shapes:
                  if shape.has_text_frame:
                      for paragraph in shape.text_frame.paragraphs:
                          for run in paragraph.runs:
                              run.text = run.text.replace(run.text,translated_list[i]).replace("\\n","\n").replace("\\'","\'").replace("\\u200b","").replace("\\xa0","").replace("\\t","  ")
                              i += 1
          elif shape.shape_type == MSO_SHAPE_TYPE.TABLE:
              table = shape.table
              for row in table.rows:
                  for cell in row.cells:
                      cell.text = cell.text.replace(cell.text,translated_list[i]).replace("\\n","\n").replace("\\'","\'").replace("\\u200b","").replace("\\xa0","").replace("\\t","  ")
                      i += 1
    except:
        pass

  print(f"\nTotal de revisões: {failed}") if len(failed) > 0 else print(f"\nTodos os {num_slides} slides traduzidos com sucesso!")

  novo_arquivo = f"{idioma}_{file}"
  document.save(novo_arquivo)

  return novo_arquivo

### Faça o upload do arquivo .pptx 📂

In [ ]:
from google.colab import files

documentos = files.upload()

if documentos:
    nome_do_documento = list(documentos.keys())[0]
    print(f"O documento '{nome_do_documento}' foi carregado com sucesso!")
else:
    print("Nenhum documento foi carregado.")


Saving Exemplo - Projeto de Navios 2023 (reduzido).pptx to Exemplo - Projeto de Navios 2023 (reduzido).pptx
O documento 'Exemplo - Projeto de Navios 2023 (reduzido).pptx' foi carregado com sucesso!


### Forneça o idioma desejado e aguarde os Agentes fazerem o trabalho 😁
  
🚨Atenção! O programa não tem limite de slides, mas a **API key gratuita** não permite muitas requisições. Consegui no máximo 15 slides.

In [ ]:
idioma = "Inglês"
documento_traduzido = processar_pptx(nome_do_documento, idioma)

Arquivo pptx selecionado

Traduzindo slide 1
Extraído: 12
['PROJETO DE NAVIOS ', 'GRUPO 3', 'ENGENHEIROS', 'Breno Guedes Gallo', 'Marcos Vicente Soares Pifano', 'Thiago Aguiar Lima', 'Poli', 'marine', ' PETROLEIRO', 'São Paulo, 20 de ', 'Outubro', ' de 2023']
Traduzido: 12
['SHIP DESIGN ', 'GROUP 3', 'ENGINEERS', 'Breno Guedes Gallo', 'Marcos Vicente Soares Pifano', 'Thiago Aguiar Lima', 'Poli', 'marine', ' OIL TANKER', 'São Paulo, 20th of ', 'October', ' of 2023']

Traduzindo slide 2
Extraído: 12
['Poli', 'marine', '1. REQUISITOS DO ARMADOR', '2. PRIMEIRAS ESTIMATIVAS', '3. PESOS E CENTROS', '4. ESPAÇO E ARRANJO', '5. ESTABILIDADE', 'AGENDA', 'PNV3415 – ', 'Projeto', ' de ', 'Navios']
Traduzido: 12
['Poli', 'marine', '1. SHIPOWNER REQUIREMENTS', '2. FIRST ESTIMATES', '3. WEIGHTS AND CENTERS', '4. SPACE AND ARRANGEMENT', '5. STABILITY', 'AGENDA', 'PNV3415 – ', 'Project', ' of ', 'Ships']

Traduzindo slide 3
Extraído: 17
['PRODUTO', 'PETRÓLEO BRUTO', 'REQUISITOS', 'ARMADOR', 'Poli', 'ma

### Prontinho! Faça o download abaixo 👇

In [ ]:
# Fazer o download do novo documento traduzido
files.download(documento_traduzido)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>